# Impacto de la deforestación en el regimen de caudales de los rios en Colombia (TEAM 28)

## Multivariate time series forecasting

Sources :

https://towardsdatascience.com/vector-autoregressions-vector-error-correction-multivariate-model-a69daf6ab618

https://towardsdatascience.com/pairs-trading-with-cryptocurrencies-e79b4a00b015

### Libraries

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.tsa.vector_ar as var
import sklearn.metrics as skm

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
figure(num = None, figsize = (15, 12), dpi = 80, facecolor = 'w', edgecolor = 'k')
plt.rcParams.update({'font.size': 16, 'figure.figsize': (15, 10), 
                     'figure.max_open_warning': 200})

# machine learning: XGB
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from xgboost.sklearn import XGBRegressor # wrapper

<Figure size 1200x960 with 0 Axes>

In [2]:
print(plt.rcParams.keys())

KeysView(RcParams({'_internal.classic_mode': False,
          'agg.path.chunksize': 0,
          'animation.avconv_args': [],
          'animation.avconv_path': 'avconv',
          'animation.bitrate': -1,
          'animation.codec': 'h264',
          'animation.convert_args': [],
          'animation.convert_path': 'convert',
          'animation.embed_limit': 20.0,
          'animation.ffmpeg_args': [],
          'animation.ffmpeg_path': 'ffmpeg',
          'animation.frame_format': 'png',
          'animation.html': 'none',
          'animation.html_args': [],
          'animation.writer': 'ffmpeg',
          'axes.autolimit_mode': 'data',
          'axes.axisbelow': 'line',
          'axes.edgecolor': 'black',
          'axes.facecolor': 'white',
          'axes.formatter.limits': [-5, 6],
          'axes.formatter.min_exponent': 0,
          'axes.formatter.offset_threshold': 4,
          'axes.formatter.use_locale': False,
          'axes.formatter.use_mathtext': False,
        

### Read Data

In [3]:
macrodata = pd.read_csv('../data/matrix/matrix_consol_v2.zip')

macrodata.head(10)

,date,year,month,mc,v_flow_mean,v_loss_cover,v_rainfall_total,v_temperature_mean
0,2000-01,2000,1,7,230.4,0.000000,334.0,NaN
1,2000-02,2000,2,7,272.4,0.000133,400.0,NaN
2,2000-03,2000,3,7,321.6,0.000265,319.0,NaN
3,2000-04,2000,4,7,310.8,0.000398,248.0,NaN
4,2000-05,2000,5,7,410.0,0.000531,302.0,NaN
5,2000-06,2000,6,7,295.9,0.000663,81.0,NaN
6,2000-07,2000,7,7,244.2,0.000796,96.0,NaN
7,2000-08,2000,8,7,255.0,0.000928,64.0,NaN
8,2000-09,2000,9,7,233.8,0.001061,262.0,NaN
9,2000-10,2000,10,7,216.0,0.001194,141.0,NaN


In [4]:
mcs = macrodata['mc'].unique()
mcs.sort()

print(mcs)

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48]


### Split data for machine learning algorithms

In [5]:
temp_df = macrodata.copy()

data_train = pd.DataFrame()
data_test = pd.DataFrame()

for i in mcs:
    #train, test = train_test_split(temp_df[temp_df['mc'] == i], test_size = 0.2)
    nobs = 24 # 10% de 240
    train, test = temp_df[temp_df['mc'] == i].iloc[0:-nobs], temp_df[temp_df['mc'] == i].iloc[-nobs:]
    data_train = pd.concat([data_train, train], axis = 0)
    data_test = pd.concat([data_test, test], axis = 0)

print('Total data')
print('----------')
print(macrodata.shape)
print(macrodata.dtypes)
print()
print('data_train')
print('----------')
print(data_train.shape)
print(data_train.dtypes)
print()
print('data_test')
print('---------')
print(data_test.shape)
print(data_test.dtypes)

temp_df.head(10)

Total data
----------
(11520, 8)
date                   object
year                    int64
month                   int64
mc                      int64
v_flow_mean           float64
v_loss_cover          float64
v_rainfall_total      float64
v_temperature_mean    float64
dtype: object

data_train
----------
(10368, 8)
date                   object
year                    int64
month                   int64
mc                      int64
v_flow_mean           float64
v_loss_cover          float64
v_rainfall_total      float64
v_temperature_mean    float64
dtype: object

data_test
---------
(1152, 8)
date                   object
year                    int64
month                   int64
mc                      int64
v_flow_mean           float64
v_loss_cover          float64
v_rainfall_total      float64
v_temperature_mean    float64
dtype: object


,date,year,month,mc,v_flow_mean,v_loss_cover,v_rainfall_total,v_temperature_mean
0,2000-01,2000,1,7,230.4,0.000000,334.0,NaN
1,2000-02,2000,2,7,272.4,0.000133,400.0,NaN
2,2000-03,2000,3,7,321.6,0.000265,319.0,NaN
3,2000-04,2000,4,7,310.8,0.000398,248.0,NaN
4,2000-05,2000,5,7,410.0,0.000531,302.0,NaN
5,2000-06,2000,6,7,295.9,0.000663,81.0,NaN
6,2000-07,2000,7,7,244.2,0.000796,96.0,NaN
7,2000-08,2000,8,7,255.0,0.000928,64.0,NaN
8,2000-09,2000,9,7,233.8,0.001061,262.0,NaN
9,2000-10,2000,10,7,216.0,0.001194,141.0,NaN


In [6]:
data_train.head(10)

,date,year,month,mc,v_flow_mean,v_loss_cover,v_rainfall_total,v_temperature_mean
7920,2000-01,2000,1,1,2.45,0.000000,299.12,18.300
7921,2000-02,2000,2,1,2.14,0.000552,333.86,18.200
7922,2000-03,2000,3,1,2.53,0.001105,264.73,18.500
7923,2000-04,2000,4,1,2.38,0.001657,170.73,18.700
7924,2000-05,2000,5,1,4.64,0.002210,65.17,18.400
7925,2000-06,2000,6,1,3.06,0.002762,72.36,18.875
7926,2000-07,2000,7,1,3.12,0.003314,42.85,18.800
7927,2000-08,2000,8,1,4.84,0.003867,52.01,19.600
7928,2000-09,2000,9,1,3.22,0.004419,20.73,18.650
7929,2000-10,2000,10,1,2.33,0.004972,92.40,18.300


In [7]:
data_test.head(10)

,date,year,month,mc,v_flow_mean,v_loss_cover,v_rainfall_total,v_temperature_mean
8136,2018-01,2018,1,1,1.60,0.072060,222.87,18.80
8137,2018-02,2018,2,1,1.57,0.072256,143.49,19.10
8138,2018-03,2018,3,1,2.10,0.072453,274.72,19.60
8139,2018-04,2018,4,1,2.99,0.072649,65.35,18.80
8140,2018-05,2018,5,1,3.40,0.072846,214.84,19.00
8141,2018-06,2018,6,1,4.61,0.073042,145.44,19.10
8142,2018-07,2018,7,1,4.19,0.073239,15.04,19.60
8143,2018-08,2018,8,1,3.46,0.073435,21.49,20.20
8144,2018-09,2018,9,1,2.31,0.073632,3.66,19.85
8145,2018-10,2018,10,1,2.48,0.073828,83.10,19.20


In [8]:
dates = pd.DataFrame(pd.date_range('2018-01-01','2019-12-31' , freq='1M') - 
             pd.offsets.MonthBegin(1))
dates.columns = ['date']

dates['year'] = pd.DatetimeIndex(dates['date']).year
dates['month'] = pd.DatetimeIndex(dates['date']).month

dates.head()

,date,year,month
0,2018-01-01,2018,1
1,2018-02-01,2018,2
2,2018-03-01,2018,3
3,2018-04-01,2018,4
4,2018-05-01,2018,5


## XGBoost

[XGBoost](https://github.com/dmlc/xgboost/blob/master/doc/model.md) is an implementation of Gradient Boosted Decision trees designed for speed and performance. Its more suitable name is a as [regularized Gradient Boosting](http://datascience.la/xgboost-workshop-and-meetup-talk-with-tianqi-chen/), as it uses a more regularized model formalization to control over-fitting.

Additional advantages of this algorythm are:
- Automated missing values handling: XGB uses a "learned" default direction for the missing values. "Learned" means learned in the tree construction process by choosing the best direction that optimizes the training loss.
- Interactive feature analysis (yet implemented only in R): plots the structure of decision trees with splits and leaves.
- Feature importance analysis: a sorted barplot of the most significant variables.

<div class = "alert alert-block alert-info"> As we already saw in the previos section our data is higly seasonal and not random (dependent). Therefore, before fitting any models we need to "smooth" target variable Sales. The typical preprocessing step is to log transform the data in question. Once we perform the forecasting we will unwind log transformations in reverse order. </div>

### Model Training

**Approach**

1. Split train data to train and test set to evaluate the model.
2. Set eta to a relatively high value (e.g. 0.05 ~ 0.1), num_round to 300 ~ 500
3. Use grid search to find the best combination of additional parameters.
4. Lower eta until we reach the optimum.
5. Use the validation set as watchlist to retrain the model with the best parameters.

In [9]:
# split into training and evaluation sets

# v_flow_mean
# v_loss_cover
# v_rainfall_total
# v_temperature_mean
# v_flow_mean_log
# v_loss_cover_log
# v_rainfall_total_log
# v_flow_mean_log_diff
# v_loss_cover_log_diff
# v_rainfall_total_log_diff

# predictors = [x for x in train_store.columns if x not in ['Customers', 'Sales', 'SalePerCustomer']]
# y = np.log(train_store.Sales) # log transformation of Sales
# X = train_store

# # split the data into train/test set
# X_train, X_test, y_train, y_test = train_test_split(X, y, 
#                                                     test_size = 0.1, # 30% for the evaluation set
#                                                     random_state = 42)

In [10]:
data_train['v_loss_cover_porc'] = data_train['v_loss_cover'] * 1000

data_train.head()

,date,year,month,mc,v_flow_mean,v_loss_cover,v_rainfall_total,v_temperature_mean,v_loss_cover_porc
7920,2000-01,2000,1,1,2.45,0.000000,299.12,18.3,0.000000
7921,2000-02,2000,2,1,2.14,0.000552,333.86,18.2,0.552391
7922,2000-03,2000,3,1,2.53,0.001105,264.73,18.5,1.104781
7923,2000-04,2000,4,1,2.38,0.001657,170.73,18.7,1.657172
7924,2000-05,2000,5,1,4.64,0.002210,65.17,18.4,2.209563


In [11]:
data_test['v_loss_cover_porc'] = data_test['v_loss_cover'] * 1000

data_test.head()

,date,year,month,mc,v_flow_mean,v_loss_cover,v_rainfall_total,v_temperature_mean,v_loss_cover_porc
8136,2018-01,2018,1,1,1.60,0.072060,222.87,18.8,72.060005
8137,2018-02,2018,2,1,1.57,0.072256,143.49,19.1,72.256450
8138,2018-03,2018,3,1,2.10,0.072453,274.72,19.6,72.452895
8139,2018-04,2018,4,1,2.99,0.072649,65.35,18.8,72.649340
8140,2018-05,2018,5,1,3.40,0.072846,214.84,19.0,72.845785


In [12]:
predictors = ['v_loss_cover_porc', 'v_rainfall_total']
# predictors = ['v_loss_cover', 'v_rainfall_total']

In [ ]:
# evaluation metric: rmspe
# Root Mean Square Percentage Error
# code chunk shared at Kaggle

def rmspe(y, yhat):
    return np.sqrt(np.mean((yhat / y-1) ** 2))

def rmspe_xg(yhat, y):
    y = np.expm1(y.get_label())
    yhat = np.expm1(yhat)
    return "rmspe", rmspe(y, yhat)

**Tuning Parameters** 

- eta: Step size used in updating weights. Lower value means slower training but better convergence.
- num_round: Total number of iterations.
- subsample: The ratio of training data used in each iteration; combat overfitting. Should be configured in the range of 30% to 80% of the training dataset, and compared to a value of 100% for no sampling.
- colsample_bytree: The ratio of features used in each iteration, default 1.
- max_depth: The maximum depth of each tree. If we do not limit max depth, gradient boosting would eventually overfit.
- early_stopping_rounds: If there's no increase in validation score for a given number of iterations, the algorithm will stop early, also combats overfitting.

### XGB with xgboost library (First simplified version just with train - Version 1)

In [ ]:
# base parameters
# params = {
#     'booster': 'gbtree', 
#     'objective': 'reg:squarederror', # regression task
#     'subsample': 0.8, # 80% of data to grow trees and prevent overfitting
#     'colsample_bytree': 0.85, # % of features used
#     'eta': 0.08, 
#     'max_depth': 10, 
#     'seed': 42} # for reproducible results

params = {
    'booster': 'gbtree', 
    'objective': 'reg:squarederror', # regression task
    'subsample': 0.8, # 80% of data to grow trees and prevent overfitting
    'colsample_bytree': 1, # % of features used
    'eta': 0.08, 
    'max_depth': 10, 
    'seed': 42} # for reproducible results

In [ ]:
# XGB with xgboost library (First simplified version just with train - Version 1)

forecast_steps = 24

XGB_metrics = pd.DataFrame()
XGB_prediction = pd.DataFrame()

df_forecast = pd.DataFrame()

for i in mcs:
    X_train = data_train[data_train['mc'] == i].copy()
    y_train = np.log(X_train.v_flow_mean[X_train['mc'] == i] + 0.01)
    X_test = data_test[data_test['mc'] == i].copy()
    y_test = np.log(X_test.v_flow_mean[data_test['mc'] == i] + 0.01)
    y_test_org = X_test.v_flow_mean[data_test['mc'] == i] + 0.01
    
    dtrain = xgb.DMatrix(X_train[predictors], y_train)
    dtest = xgb.DMatrix(X_test[predictors], y_test)

    watchlist = [(dtrain, 'train'), (dtest, 'test')]

    print("\n=====================================")
    print('MC = %s' % i)
    print("=====================================\n")
    
    xgb_model = xgb.train(params, dtrain, 300, evals = watchlist, 
                          early_stopping_rounds = 100, feval = rmspe_xg, 
                          verbose_eval = False)

#     Funcional sin tunning *********

    yhat = xgb_model.predict(xgb.DMatrix(X_test[predictors]))
    error = rmspe(X_test.v_flow_mean.values, np.exp(yhat))

    print('First validation yelds RMSPE: {:.6f}'.format(error))
    print(yhat)
    
    xgb.plot_importance(xgb_model)    
    
    # predictions to unseen data
    unseen = xgb.DMatrix(X_test[predictors])
    test_p = xgb_model.predict(unseen)
    
    temp_df = data_test[data_test['mc'] == i].copy()
    temp_df = temp_df[['v_flow_mean', 'v_loss_cover', 'v_rainfall_total']]
    temp_df.reset_index(drop=True, inplace=True)
    
    forecast = pd.DataFrame({'v_flow_mean_mean': np.exp(test_p)})
    forecast = pd.concat([dates, forecast, temp_df], axis = 1)
    forecast['mc'] = i
    
    print(forecast.head())
    
    forecast_errors = [temp_df.v_flow_mean.iloc[j] - forecast.v_flow_mean_mean.iloc[j] 
                       for j in range(forecast_steps)]
    bias = sum(forecast_errors) * 1.0 / (forecast_steps)
#     print('Bias : %f' % bias)

    mae = skm.mean_absolute_error(temp_df.v_flow_mean, forecast.v_flow_mean_mean)
#     print('MAE : %f' % mae)

    mse = skm.mean_squared_error(temp_df.v_flow_mean, forecast.v_flow_mean_mean)
    rmse = np.sqrt(mse)
#     print('MSE : %f' % mse)
#     print('RMSE : %f' % rmse) 
    
    resultados = [i, bias, mae, mse, rmse]
    resultados = pd.DataFrame([resultados], columns = ['mc', 'Bias', 'MAE', 'MSE', 'RMSE'])
    
    print(resultados.head())
    print('===========================================================\n')

    forecast = forecast[['date', 'year', 'month', 'mc', 'v_flow_mean_mean', 
                         'v_flow_mean', 'v_loss_cover', 'v_rainfall_total']]
    
    XGB_metrics = pd.concat([XGB_metrics, resultados], axis = 0)
    XGB_prediction = pd.concat([XGB_prediction, forecast], axis = 0)
    
#     print(y_test_org.head())

#     ******************************************************************
    
#     Prueba ******
    
#     params_sk = {'max_depth': 10, 
#             'n_estimators': 100, # the same as num_rounds in xgboost
#             'objective': 'reg:squarederror', 
#             'subsample': 0.8, 
#             'colsample_bytree': 0.85, 
#             'learning_rate': 0.025, 
#             'seed': 42}     

#     skrg = XGBRegressor(**params_sk)

#     skrg.fit(X_train[predictors], y_train, 
#              eval_set = [(X_train[predictors], y_train), (X_test[predictors], y_test)])

#     results = skrg.evals_result()
#     epochs = len(results['validation_0']['rmse'])
#     x_axis = range(0, epochs)
#     # plot AUC
#     fig, ax = plt.subplots()
#     ax.plot(x_axis, results['validation_0']['rmse'], label='Train')
#     ax.plot(x_axis, results['validation_1']['rmse'], label='Test')
#     ax.legend()
#     plt.ylabel('RMSE')
#     plt.title('XGBoost RMSE')
#     plt.show()
        
    
#     ******************************************************************
    

# Guardar en archivos

XGB_metrics.to_csv('../model/XGB_results_v1.csv', index = False)
XGB_metrics.head()

XGB_prediction['v_flow_mean_mean'] = XGB_prediction['v_flow_mean_mean'].apply(lambda x: 
                                                                              0.01 if x <= 0 
                                                                              else x)
XGB_prediction.to_csv('../model/XGB_predictions_v1.csv', index = False)

XGB_prediction.head()
 

-------------------------------

### XGB with xgboost library (First simplified version just with train - Version 1) - Prediction 2020 - 2021

In [13]:
scenarios = pd.read_excel('../data/matrix/Esc_Predicciones_longitudinal.xlsx')

scenarios['v_flow_mean'] = 0

scenarios = scenarios[['date', 'mc', 'v_flow_mean', 'v_loss_cover', 'v_rainfall_total', 
                       'scenario']]

scenarios.head(10)

,date,mc,v_flow_mean,v_loss_cover,v_rainfall_total,scenario
0,2020-01-01,1,0,0.07693,210.44,Def_25%_P_A1
1,2020-02-01,1,0,0.07702,135.49,Def_25%_P_A1
2,2020-03-01,1,0,0.07710,259.40,Def_25%_P_A1
3,2020-04-01,1,0,0.07718,61.71,Def_25%_P_A1
4,2020-05-01,1,0,0.07726,202.86,Def_25%_P_A1
5,2020-06-01,1,0,0.07735,137.33,Def_25%_P_A1
6,2020-07-01,1,0,0.07743,14.20,Def_25%_P_A1
7,2020-08-01,1,0,0.07751,20.29,Def_25%_P_A1
8,2020-09-01,1,0,0.07759,3.46,Def_25%_P_A1
9,2020-10-01,1,0,0.07768,78.47,Def_25%_P_A1


In [14]:
dates = pd.DataFrame(pd.date_range('2020-01-01','2021-12-31' , freq='1M') - 
             pd.offsets.MonthBegin(1))
dates.columns = ['date']

dates['year'] = pd.DatetimeIndex(dates['date']).year
dates['month'] = pd.DatetimeIndex(dates['date']).month

dates.head()

,date,year,month
0,2020-01-01,2020,1
1,2020-02-01,2020,2
2,2020-03-01,2020,3
3,2020-04-01,2020,4
4,2020-05-01,2020,5


In [15]:
# evaluation metric: rmspe
# Root Mean Square Percentage Error
# code chunk shared at Kaggle

def rmspe(y, yhat):
    return np.sqrt(np.mean((yhat / y-1) ** 2))

def rmspe_xg(yhat, y):
    y = np.expm1(y.get_label())
    yhat = np.expm1(yhat)
    return "rmspe", rmspe(y, yhat)

In [18]:
# base parameters
# params = {
#     'booster': 'gbtree', 
#     'objective': 'reg:squarederror', # regression task
#     'subsample': 0.8, # 80% of data to grow trees and prevent overfitting
#     'colsample_bytree': 0.85, # % of features used
#     'eta': 0.08, 
#     'max_depth': 10, 
#     'seed': 42} # for reproducible results

params = {
    'booster': 'gbtree', 
    'objective': 'reg:squarederror', # regression task
    'subsample': 0.8, # 80% of data to grow trees and prevent overfitting
    'colsample_bytree': 1, # % of features used
    'eta': 0.08, 
    'max_depth': 10, 
    'seed': 42} # for reproducible results

forecast_steps = 24
predictors = ['v_loss_cover_porc', 'v_rainfall_total']
escen = scenarios['scenario'].unique()

XGB_prediction = pd.DataFrame()

data_train_predict = macrodata.copy()
data_train_predict['v_loss_cover_porc'] = data_train_predict['v_loss_cover'] * 1000

for j in escen:
    
    data_test_predict = scenarios[scenarios['scenario'] == j].copy()
    data_test_predict['v_loss_cover_porc'] = data_test_predict['v_loss_cover'] * 1000
    
    print('\n==================================')
    print('Escenario :', j)
    print('==================================\n')

    for i in mcs:
        X_train = data_train_predict[data_train_predict['mc'] == i].copy()
        y_train = np.log(X_train.v_flow_mean[X_train['mc'] == i] + 0.01)
        X_test = data_test_predict[data_test_predict['mc'] == i].copy()
        y_test = np.log(X_test.v_flow_mean[data_test_predict['mc'] == i] + 0.01)
        y_test_org = X_test.v_flow_mean[data_test_predict['mc'] == i] + 0.01

        dtrain = xgb.DMatrix(X_train[predictors], y_train)
        dtest = xgb.DMatrix(X_test[predictors], y_test)

        watchlist = [(dtrain, 'train'), (dtest, 'test')]

        xgb_model = xgb.train(params, dtrain, 300, evals = watchlist, 
                              early_stopping_rounds = 100, feval = rmspe_xg, 
                              verbose_eval = False)

    #     Funcional sin tunning *********

        yhat = xgb_model.predict(xgb.DMatrix(X_test[predictors]))
#         error = rmspe(X_test.v_flow_mean.values, np.exp(yhat))

#         print('First validation yelds RMSPE: {:.6f}'.format(error))
#         print(yhat)

#         xgb.plot_importance(xgb_model)    

        # predictions to unseen data
        unseen = xgb.DMatrix(X_test[predictors])
        test_p = xgb_model.predict(unseen)

        temp_df = data_test_predict[data_test_predict['mc'] == i].copy()
        temp_df = temp_df[['v_flow_mean', 'v_loss_cover', 'v_rainfall_total']]
        temp_df.reset_index(drop=True, inplace=True)

        forecast = pd.DataFrame({'v_flow_mean_forecast': np.exp(test_p)})
        forecast = pd.concat([dates, forecast, temp_df], axis = 1)
        forecast['mc'] = i

        print('\n ===================================================== \n')
        
        print(forecast.head())

        forecast = forecast[['date', 'year', 'month', 'mc', 'v_flow_mean_forecast', 
                             'v_flow_mean', 'v_loss_cover', 'v_rainfall_total']]
        
        forecast['scenario'] = j
        
        print(j, i, forecast.shape, forecast.v_flow_mean_forecast.mean(), 
              np.exp(y_train).mean(), 
              forecast.v_loss_cover.mean(), forecast.v_rainfall_total.mean())

        XGB_prediction = pd.concat([XGB_prediction, forecast], axis = 0)

    XGB_prediction.head()



Escenario : Def_25%_P_A1



        date  year  month  v_flow_mean_forecast  v_flow_mean  v_loss_cover  \
0 2020-01-01  2020      1              3.338292            0       0.07693   
1 2020-02-01  2020      2              2.997712            0       0.07702   
2 2020-03-01  2020      3              2.966321            0       0.07710   
3 2020-04-01  2020      4              2.462032            0       0.07718   
4 2020-05-01  2020      5              3.353937            0       0.07726   

   v_rainfall_total  mc  
0            210.44   1  
1            135.49   1  
2            259.40   1  
3             61.71   1  
4            202.86   1  
Def_25%_P_A1 1 (24, 9) 2.7960474 2.881833333333333 0.0778825 122.51666666666665


        date  year  month  v_flow_mean_forecast  v_flow_mean  v_loss_cover  \
0 2020-01-01  2020      1             35.982273            0       0.06117   
1 2020-02-01  2020      2             12.097168            0       0.06123   
2 2020-03-01  2020      3     



        date  year  month  v_flow_mean_forecast  v_flow_mean  v_loss_cover  \
0 2020-01-01  2020      1             79.372520            0       0.01751   
1 2020-02-01  2020      2             42.312725            0       0.01753   
2 2020-03-01  2020      3             72.991280            0       0.01754   
3 2020-04-01  2020      4             68.090424            0       0.01756   
4 2020-05-01  2020      5            119.315605            0       0.01758   

   v_rainfall_total  mc  
0             94.94  14  
1             24.17  14  
2            188.52  14  
3            215.72  14  
4            330.81  14  
Def_25%_P_A1 14 (24, 9) 72.02181 76.75396074218725 0.01771 157.24916666666664


        date  year  month  v_flow_mean_forecast  v_flow_mean  v_loss_cover  \
0 2020-01-01  2020      1             26.749525            0       0.20571   
1 2020-02-01  2020      2             21.499210            0       0.20593   
2 2020-03-01  2020      3             28.161438            



        date  year  month  v_flow_mean_forecast  v_flow_mean  v_loss_cover  \
0 2020-01-01  2020      1             27.938030            0       0.06874   
1 2020-02-01  2020      2             12.827092            0       0.06881   
2 2020-03-01  2020      3             12.913488            0       0.06888   
3 2020-04-01  2020      4             45.937153            0       0.06895   
4 2020-05-01  2020      5             43.910862            0       0.06903   

   v_rainfall_total  mc  
0            168.57  26  
1             59.76  26  
2             83.16  26  
3            374.20  26  
4            235.56  26  
Def_25%_P_A1 26 (24, 9) 31.353083 68.90254166666666 0.06956999999999999 170.33666666666667


        date  year  month  v_flow_mean_forecast  v_flow_mean  v_loss_cover  \
0 2020-01-01  2020      1             11.773438            0       0.04434   
1 2020-02-01  2020      2              5.440589            0       0.04439   
2 2020-03-01  2020      3              9.41334

<ipython-input-15-5beb244f18bf>:6: RuntimeWarning: divide by zero encountered in true_divide
  return np.sqrt(np.mean((yhat / y-1) ** 2))




        date  year  month  v_flow_mean_forecast  v_flow_mean  v_loss_cover  \
0 2020-01-01  2020      1            229.243103            0       0.03712   
1 2020-02-01  2020      2            259.038757            0       0.03716   
2 2020-03-01  2020      3            323.976868            0       0.03719   
3 2020-04-01  2020      4            422.988556            0       0.03723   
4 2020-05-01  2020      5            678.989807            0       0.03726   

   v_rainfall_total  mc  
0            105.98  34  
1            171.38  34  
2            219.72  34  
3            276.36  34  
4            381.23  34  
Def_25%_P_A1 34 (24, 9) 370.31348 380.7276405872594 0.037524999999999996 228.2658333333333


        date  year  month  v_flow_mean_forecast  v_flow_mean  v_loss_cover  \
0 2020-01-01  2020      1            362.592621            0       0.09518   
1 2020-02-01  2020      2            180.540787            0       0.09527   
2 2020-03-01  2020      3            362.59262

...



        date  year  month  v_flow_mean_forecast  v_flow_mean  v_loss_cover  \
0 2020-01-01  2020      1              2.954882            0       0.01996   
1 2020-02-01  2020      2              2.800316            0       0.02005   
2 2020-03-01  2020      3              7.033878            0       0.02014   
3 2020-04-01  2020      4              8.939735            0       0.02023   
4 2020-05-01  2020      5              6.957726            0       0.02032   

   v_rainfall_total  mc  
0            109.05  19  
1             77.52  19  
2            166.52  19  
3            250.13  19  
4            232.88  19  
Def_100%_P_B2 19 (24, 9) 4.799908 10.718401712962844 0.020995 118.82208333333334


        date  year  month  v_flow_mean_forecast  v_flow_mean  v_loss_cover  \
0 2020-01-01  2020      1             46.160309            0       0.06866   
1 2020-02-01  2020      2             47.264809            0       0.06895   
2 2020-03-01  2020      3             53.336731         



        date  year  month  v_flow_mean_forecast  v_flow_mean  v_loss_cover  \
0 2020-01-01  2020      1              5.499410            0       0.03305   
1 2020-02-01  2020      2              8.374413            0       0.03319   
2 2020-03-01  2020      3             10.800091            0       0.03333   
3 2020-04-01  2020      4              7.961928            0       0.03347   
4 2020-05-01  2020      5             10.613273            0       0.03361   

   v_rainfall_total  mc  
0            166.56  30  
1            248.20  30  
2            210.04  30  
3            238.67  30  
4            191.79  30  
Def_100%_P_B2 30 (24, 9) 10.535404 11.295887563621454 0.03466 187.1641666666667


        date  year  month  v_flow_mean_forecast  v_flow_mean  v_loss_cover  \
0 2020-01-01  2020      1              6.524251            0       0.02750   
1 2020-02-01  2020      2              6.274892            0       0.02762   
2 2020-03-01  2020      3              7.855925          

<ipython-input-15-5beb244f18bf>:6: RuntimeWarning: divide by zero encountered in true_divide
  return np.sqrt(np.mean((yhat / y-1) ** 2))




        date  year  month  v_flow_mean_forecast  v_flow_mean  v_loss_cover  \
0 2020-01-01  2020      1              4.888721            0       0.02667   
1 2020-02-01  2020      2              5.250538            0       0.02678   
2 2020-03-01  2020      3              4.899365            0       0.02689   
3 2020-04-01  2020      4              4.435529            0       0.02700   
4 2020-05-01  2020      5              4.104155            0       0.02711   

   v_rainfall_total  mc  
0            330.06  33  
1            275.01  33  
2            367.26  33  
3            181.46  33  
4            192.91  33  
Def_100%_P_B2 33 (24, 9) 4.0154123 4.026666666666666 0.027934999999999998 203.94583333333335


        date  year  month  v_flow_mean_forecast  v_flow_mean  v_loss_cover  \
0 2020-01-01  2020      1            219.568558            0       0.03723   
1 2020-02-01  2020      2            270.816620            0       0.03737   
2 2020-03-01  2020      3            310.030



        date  year  month  v_flow_mean_forecast  v_flow_mean  v_loss_cover  \
0 2020-01-01  2020      1            349.064117            0       0.18500   
1 2020-02-01  2020      2            176.815033            0       0.18577   
2 2020-03-01  2020      3            798.139160            0       0.18654   
3 2020-04-01  2020      4            834.461914            0       0.18731   
4 2020-05-01  2020      5            816.491577            0       0.18808   

   v_rainfall_total  mc  
0             75.65  45  
1             50.11  45  
2            284.91  45  
3            497.12  45  
4            436.21  45  
Def_100%_P_B2 45 (24, 9) 645.13824 775.0035364583315 0.193855 258.1829166666667


        date  year  month  v_flow_mean_forecast  v_flow_mean  v_loss_cover  \
0 2020-01-01  2020      1            862.391174            0       0.14658   
1 2020-02-01  2020      2            352.861633            0       0.14718   
2 2020-03-01  2020      3           1003.344910          

In [19]:
XGB_prediction['v_flow_mean_forecast'] = XGB_prediction['v_flow_mean_forecast'].apply(lambda x: 
                                                                              0.01 if x <= 0 
                                                                              else x)
XGB_prediction.to_csv('../model/XGB_forecast_2020_2021.csv', index = False)

------------------------